In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import os
import env
import wrangle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'env'

In [1]:
def acquire_zillow(use_cache=True):
    
    filename = ('zillow.csv')
    if os.path.exists(filename) and use_cache:
        print('Using CSV')
        return pd.read_csv(filename)
    
    print('Acquiring from Database')
    url = env.get_db_url('zillow')

    zillow = pd.read_sql('''
    SELECT *
    FROM properties_2017
    JOIN (SELECT parcelid as pid, MAX(transactiondate) as maxdate FROM predictions_2017 GROUP BY parcelid) as last_date
    ON last_date.pid = parcelid
    LEFT JOIN (SELECT parcelid as pid, transactiondate as maxdate, logerror FROM predictions_2017) as log
    ON last_date.pid = log.pid AND last_date.maxdate = log.maxdate
    LEFT JOIN propertylandusetype
    USING(propertylandusetypeid)
    LEFT JOIN storytype
    USING(storytypeid)
    LEFT JOIN typeconstructiontype
    USING(typeconstructiontypeid)
    LEFT JOIN airconditioningtype
    USING(airconditioningtypeid)
    LEFT JOIN architecturalstyletype
    USING(architecturalstyletypeid)
    LEFT JOIN buildingclasstype
    USING(buildingclasstypeid)
    LEFT JOIN heatingorsystemtype
    USING(heatingorsystemtypeid)
    ''',url)

    print('Saving to CSV')
    zillow.to_csv('zillow.csv',index=False)
    return zillow

In [2]:
def initial_look(df):
    print(f'Shape:\n\n{df.shape}\n\n')
    print(f'Describe:\n\n{df.describe(include="all")}\n\n')
    print(f'Info:\n\n{df.info()}\n\n')
    print(f'Histograms:\n\n{df.hist(figsize=(40,20), bins =20), plt.show()}')

In [3]:
def missing_rows(df):
    return pd.concat([
           df.isna().sum().rename('count'),
           df.isna().mean().rename('percent')
           ], axis=1)

In [4]:
def missing_columns(df):
    col_missing = pd.concat([
    df.isna().sum(axis=1).rename('num_cols_missing'),
    df.isna().mean(axis=1).rename('pct_cols_missing'),
    ], axis=1).value_counts().sort_index()
    col_missing = pd.DataFrame(col_missing)
    col_missing.rename(columns={0:'num_rows'},inplace=True)
    return col_missing.reset_index()

In [5]:
def handle_missing_values(df,required_col,required_row):
    required_row = round(df.shape[1] * required_row)
    required_col = round(df.shape[0] * required_col)
    df.dropna(axis=0, thresh=required_row, inplace=True)
    df.dropna(axis=1, thresh=required_col, inplace=True)
    return df

In [6]:
def cleanup_zillow(df):
    df.drop(columns=['id','pid','maxdate'],inplace=True)
    df.dropna(subset=['latitude','longitude'],inplace=True)
    df = df[df['propertylandusetypeid'].isin([260,261,263,264,266,267,269])]
    df = df[~df['unitcnt'].isin([2.0,3.0,4.0,6.0])]
    handle_missing_values(df,0.5,0.5)
    df.dropna(inplace=True)
    return df

In [ ]:
def wrangle_zillow(df,col,row):
        
        df = handle_missing_values(df,col,row)
        df = cleanup_zillow(df)
        

In [ ]:
def remove_outliers(df, column_list):
    ''' remove outliers from dataframe 
        then return the new dataframe
    '''
    # Iterate through column_list
    for col in column_list:
        
        # find percentiles
        q_25 = np.percentile(df[col], 25)
        q_75 = np.percentile(df[col], 75)
        
        # Calculate IQR
        iqr = q_75 - q_25
        
        # assign upper bound
        upper_bound = q_75 + 1.5 * iqr   
        
        # assign lower bound 
        lower_bound = q_25 - 1.5 * iqr   

        # assign df without outliers
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    # return dataframe without outliers    
    return df

In [ ]:
def split_df(df):
    '''
    This funciton splits the dataset for modeling into:
    train - for exploring the data, and fitting the models
    validate - for ensuring the model is not overfit
    test - for testing the model on unseen data
    '''
    # This seperates out the test data from the train and validate data. Test makes up 20 % of the data.
    train_validate, test = train_test_split(df, random_state=1729, test_size=0.2)
    
    # This seperates out the train and validates sets. Train makes up 56 % of the data and Validate makes up 24 %.
    train, validate = train_test_split(train_validate, random_state=1729, test_size=0.3)
    
    # The funciton returns the split sets
    return train, validate, test

In [ ]:
def encode_mall_cat(df):
    dummy_name = pd.get_dummies(df[['gender']])
    df = pd.concat([df,dummy_name],axis=1) 
    return df

In [ ]:
def scale_mall(df):

    col = ['age','annual_income','spending_score']

    df_scaled = df[col]

    minmax = MinMaxScaler()
    minmax.fit(df[col])

    df_scaled[col] = minmax.transform(df[col])
    
    return df_scaled